In [1]:
import plotly.express as px
import plotly.graph_objs as go
import json
import os
import pandas as pd

In [2]:
import plotly.io as pio
pio.renderers.default = "browser"

In [3]:
path = 'C:/Users/WB585318/WBG/Javier Romero - Panama/Pre-SCD/'
data = path+'02 data/maps'
os.chdir(data)



print(data)

C:/Users/WB585318/WBG/Javier Romero - Panama/Pre-SCD/02 data/maps


In [4]:
panama_corr = json.load(open("PAN_corr.geojson","r",encoding='ISO-8859-1'))

In [4]:
panama_distr = json.load(open("distritos-panama.geojson","r",encoding='utf8'))

In [4]:
panama_prov = json.load(open("provincias-panama.geojson","r", encoding='utf8'))

In [5]:
panama_prov['features'][13]['properties']

{'OBJECTID': 1278,
 'Shape_Leng': 171222.408993,
 'Shape_Area': 971716556.89,
 'CODPROV': '13',
 'NOMBRE': 'Comarca Kuna de Wargandí',
 'AREA': 971.717}

In [6]:
### creo id por provincias
prov_id_map = {}
for feature in panama_prov['features']:
    feature['ids'] = feature['properties']['CODPROV']
    prov_id_map[feature['properties']['NOMBRE']] = feature['ids']

In [5]:
### creo id por corregimientos 
corr_id_map = {}
for feature in panama_prov['features']:
    feature['ids'] = feature['properties']['Codigo_u']
    corr_id_map[feature['properties']['Nombre']] = feature['ids']

In [190]:
corr_id_map_1 = {}
for geometries in panama_corr['objects']['collection']['geometries']:
    geometries['ids'] = geometries['properties']['COD_CORR']
    corr_id_map_1[geometries['properties']['CORR_NOMB']] = geometries['ids']

In [7]:
prov_id_map

{'Bocas del Toro': '01',
 'Coclé': '02',
 'Comarca Emberá': '11',
 'Comarca Kuna Yala': '14',
 'Comarca Ngöbe Buglé': '12',
 'Herrera': '06',
 'Los Santos': '07',
 'Colón': '03',
 'Veraguas': '09',
 'Chiriquí': '04',
 'Comarca Kuna de Madungandí': '10',
 'Panamá': '08',
 'Darién': '05',
 'Comarca Kuna de Wargandí': '13'}

In [5]:
### creo id por distritos
distr_id_map = {}
for feature in panama_distr['features']:
    feature['ids'] = feature['properties']['DIST_CODIG']
    distr_id_map[feature['properties']['DIST_NOMBR']] = feature['ids']

In [48]:
df = pd.read_csv("Povertymap_2011_2015.csv")
df['ids'] = df['corr_nom'].apply(lambda x:corr_id_map[x]) 
df['extreme_pov'] = df['av_FGT0e_15']
df['poverty'] = df['av_FGT0g_15']

In [24]:
df1 = pd.read_csv("Povertymap_2015_distr_climate.csv")
df1['ids'] = df1['dist_nom'].apply(lambda x:distr_id_map[x]) 
df1['extreme_pov'] = df1['e_poor_dist_15']
df1['poverty'] = df1['g_poor_dist_15']

In [9]:
df2 = pd.read_csv("Povertymap_2015_2019_prov.csv")
df2['ids'] = df2['provincia '].apply(lambda x:prov_id_map[x]) 
df2['extreme_pov'] = df2['pov320_2019']
df2['extreme_pov_var'] = df2['var320porc2015-2019']
df2['poverty'] = df2['pov550_2019']
df2['poverty_var'] = df2['var_poor550_2015-2019']

In [42]:
fig = px.choropleth(df,locations='ids',geojson=panama_corr, featureidkey='properties.Codigo_u', color='poverty')
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [8]:
## poverty by districts 

fig = px.choropleth(df1,locations='ids',
                    geojson=panama_distr, 
                    featureidkey='properties.CODPROV', 
                    color='poverty',
                    hover_name='dist_nom',
                   color_continuous_scale="Viridis",
                   labels='dist_nom')
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [36]:
## vulnerability to climate change 

fig = px.choropleth(df1,locations='ids',
                    geojson=panama_distr, 
                    featureidkey='properties.DIST_CODIG', 
                    color='Color',
                    hover_name='dist_nom',
                   color_continuous_scale="Viridis",
                    color_discrete_map={'High0':'#D62728', 
                                        'High1':'#9D755D',
                                        'High2':'rgb(102,102,102)',
                                        'Medium0':'#E48F72',
                                        'Medium1':'#BAB0AC',
                                        'Medium2':'#72B7B2',
                                        'Low0':'rgb(254,217,166)',
                                        'Low1':'rgb(141,211,199)',
                                        'Low2':'rgb(56,166,165)',
                                        'Less of 25% rural':'#E2E2E2'},
                   labels='dist_nom')
fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(colorbar_orientation='h')
fig.show()

In [10]:
## poverty by provinces
fig = px.choropleth(df2,locations='ids',
                    geojson=panama_prov, 
                    featureidkey='properties.CODPROV', 
                    color='poverty',
                    hover_name='provincia ',
                   color_continuous_scale='blues',
                   labels='provincia')
                   #range_color=[1,70])
fig2 = px.scatter_geo(df2,locations='ids',
                    geojson=panama_prov, 
                    featureidkey='properties.CODPROV', 
                    color='poverty_var',
                    hover_name='provincia ',
                   color_continuous_scale='reds',
                     size='var550porc2015-2019_abs',
                   labels='provincia')
fig.add_trace(fig2.data[0], )
#fig.update_traces(marker_line_color="white")
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(coloraxis = {'colorscale':'viridis'})
fig.show()

In [34]:
## poverty by provinces
fig = px.choropleth(df2,locations='ids',
                    geojson=panama_prov, 
                    featureidkey='properties.CODPROV', 
                    color='poverty',
                    hover_name='provincia ',
                   color_continuous_scale='blues',
                   labels='provincia')
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(coloraxis = {'colorscale':'viridis'})
fig.show()

In [47]:
fig = px.scatter_geo(df2,locations='ids',
                    geojson=panama_prov, 
                    featureidkey='properties.CODPROV', 
                    color='poverty_var',
                    hover_name='provincia ',
                   color_continuous_scale='blues',
                     size='var550porc2015-2019_abs',
                   labels='provincia')
fig.update_geos(fitbounds="locations", visible=True)
fig.show()
